<a href="https://colab.research.google.com/github/emirhansarioglu/Cross-Modal-Learning/blob/main/Thesis_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Modal Learning for Scene Classification in Remote Sensing: Thesis Code

## Imports

In [ ]:
!pip install rasterio
!pip install tqdm
import tqdm
import rasterio
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.metrics import f1_score
import zipfile
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(torch.__version__)

## Data Unzip


In [ ]:
train_zip_path = '/content/drive/My Drive/Bachelor_Thesis/Dataset/Train_dataset.zip'
test_zip_path = '/content/drive/My Drive/Bachelor_Thesis/Dataset/Test_dataset.zip'

with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

## Custom Dataset


In [ ]:
class SentDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels_frame.iloc[idx, 0]+'.tif')
        with rasterio.open(img_name) as src:
                image = src.read()
                image = np.transpose(np.array(image), (1,2,0))

        labels = self.labels_frame.iloc[idx, 3:].values.astype('float')
        if self.transform:
            image = self.transform(image)
            image = image.float()
            s2_image = image[2:]
            s1_image = image[:2]
            labels = torch.tensor(labels, dtype=torch.float32)
        return s2_image, s1_image, labels

train_csv = '/content/Train_dataset/labels.csv'
train_dir = '/content/Train_dataset/images'
test_csv = '/content/Test_dataset/labels.csv'
test_dir = '/content/Test_dataset/images'
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
Traindataset = SentDataset(csv_file=train_csv, root_dir=train_dir, transform=transform)
Trainloader = DataLoader(Traindataset, batch_size=32, shuffle=True)
Testdataset = SentDataset(csv_file=test_csv, root_dir=test_dir, transform=transform)
Testloader = DataLoader(Testdataset, batch_size=32, shuffle=False)

## NN Models

In [ ]:
'''
This block includes models that will be used by networks
s1_feature_extraction and s2_feature_extraction are modality specific feature vector generators
individual_classification model will be used in the cross-modal-focal-loss structure 2 times, 1 for each modality
downstream_classfication is the shared part of the network
'''

class s1_feature_extraction(nn.Module):
    def __init__(self, feature_vec_dim = 2048):
        super(s1_feature_extraction, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(256 * 7 * 7, feature_vec_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 256 * 7 * 7)
        x = self.fc(x)
        x = self.relu(x)
        return x

class s2_feature_extraction(nn.Module):
    def __init__(self, feature_vec_dim = 2048):
        super(s2_feature_extraction, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(256 * 7 * 7, feature_vec_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.relu(x)
        x = x.view(-1, 256 * 7 * 7)
        x = self.fc(x)
        x = self.relu(x)
        return x

class individual_classification(nn.Module):
    def __init__(self, shared_vec_space_dim=2048):
        super(individual_classification, self).__init__()
        self.fc1 = nn.Linear(shared_vec_space_dim, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 19)
        self.relu = nn.ReLU()

    def forward(self, feature_s1):
        x = self.fc1(feature_s1)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = torch.sigmoid(x)
        return x

class downstream_classification(nn.Module):
    def __init__(self, shared_vec_space_dim=2048):
        super(downstream_classification, self).__init__()
        self.relu = nn.ReLU()
        self.dropout_fc = nn.Dropout(0.3)
        self.fc0_5 = nn.Linear(shared_vec_space_dim, shared_vec_space_dim)
        self.fc1 = nn.Linear(shared_vec_space_dim, 1024)
        self.fc1_5 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.fc7 = nn.Linear(32, 16)
        self.fc8 = nn.Linear(16, 19)

    def forward(self, x):
        x = self.fc0_5(x)
        x = self.relu(x)
        x = self.dropout_fc(x)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc1_5(x)
        x = self.relu(x)
        x = self.dropout_fc(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.relu(x)

        x = self.fc5(x)
        x = self.relu(x)

        x = self.fc6(x)
        x = self.relu(x)

        x = self.fc7(x)
        x = self.relu(x)

        x = self.fc8(x)

        return x

## Helper Functions

In [ ]:
'''
This block contains all helper functions.

normalize_to_rgb: range values of array to [0,1] to use matplotlib.imshow
init_weights: can be used before training for better convergence
cmfl_loss: p and q are the distance of probabilities to wrong label values. Function is assymetric.

'''

def normalize_to_rgb(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

def cmfl_loss(p, q, gamma=3.0):
    w = q * (2 * p * q) / (p + q)
    loss = -1 * (1 - w) ** gamma * torch.log(p)
    return loss.mean()

def visualize_s2_sample_prediction(Testdataset, device, sentinel2_model, downstream_model, ix):
    s2_sample, s1_sample, label_sample = Testdataset[ix]
    label_sample = label_sample.to('cpu')
    s2_sample = s2_sample.unsqueeze(0).to(device)
    feature_s2 = sentinel2_model(s2_sample)
    s2_downstream_logits = downstream_model(feature_s2)
    prediction = np.array((torch.sigmoid(s2_downstream_logits) > 0.3).float().to('cpu')).squeeze()
    short_labels = np.array(["Urban fabric", "Industrial units", "Arable land", "Permanent crops", "Pastures", "Complex Cultivation", "Agriculture",
                    "Agro-forestry", "Broad-leaved", "Coniferous forest", "Mixed forest", "Grassland", "Moors, Heathland", "Woodland", "Beaches",
                    "Inland wetlands", "Coastal wetlands", "Inland waters", "Marine waters"])
    predicted_labels = short_labels[prediction == 1]
    true_labels = short_labels[np.array(label_sample) == 1]

    true_positives = [label for label in predicted_labels if label in true_labels]
    false_positives = [label for label in predicted_labels if label not in true_labels]
    false_negatives = [label for label in true_labels if label not in predicted_labels]

    title_parts = []

    for label in true_positives:
        title_parts.append((label, 'green'))
    for label in false_positives:
        title_parts.append((label, 'red'))
    for label in false_negatives:
        title_parts.append((label, 'blue'))

    s2_sample = s2_sample.squeeze().to('cpu')
    s2_rgb = np.dstack(np.array([s2_sample[3], s2_sample[2], s2_sample[1]]))

    plt.imshow(normalize_to_rgb(s2_rgb))
    plt.axis('off')

    y_offset = -0.1
    for label in true_positives:
        plt.text(0.5, y_offset, label, color='green', fontsize=12, ha='center', va='bottom', transform=plt.gca().transAxes)
        y_offset -= 0.05

    for label in false_positives:
        plt.text(0.5, y_offset, label, color='red', fontsize=12, ha='center', va='bottom', transform=plt.gca().transAxes)
        y_offset -= 0.05

    for label in false_negatives:
        plt.text(0.5, y_offset, label, color='blue', fontsize=12, ha='center', va='bottom', transform=plt.gca().transAxes)
        y_offset -= 0.05

    plt.show()


## Training and Evaluation Functions

In [ ]:
def train_cmfl_model_with_val(sentinel2_model, sentinel1_model,
                         s1_classifier, s2_classifier, downstream_model, optimizer,
                         dataset, device, val_split=0.2, num_epochs=18, batch_size=32):

    bce_loss_fn = nn.BCEWithLogitsLoss()
    train_losses = []
    val_losses = []
    dataset_size = len(dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(num_epochs):
        # TRAINING
        sentinel2_model.train()
        sentinel1_model.train()
        downstream_model.train()
        s1_classifier.train()
        s2_classifier.train()
        epoch_loss = 0
        for batch in tqdm(train_loader):
            images_s2, images_s1, labels = batch
            images_s2 = images_s2.to(device)
            images_s1 = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            feature_s1 = sentinel1_model(images_s1)
            s1_classifier_preds = s1_classifier(feature_s1)
            s1_downstream_logits = downstream_model(feature_s1)
            s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
            feature_s2 = sentinel2_model(images_s2)
            s2_classifier_preds = s2_classifier(feature_s2)
            s2_downstream_logits = downstream_model(feature_s2)
            s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
            shared_network_loss = s1_downstream_loss + s2_downstream_loss
            s1 = torch.where(labels == 1, s1_classifier_preds, 1 - s1_classifier_preds)
            s2 = torch.where(labels == 1, s2_classifier_preds, 1 - s2_classifier_preds)
            cmfl = cmfl_loss(s1, s2) + cmfl_loss(s2, s1)
            loss = (shared_network_loss + cmfl)/2
            loss.backward()

            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= 2*len(train_loader.dataset)
        train_losses.append(epoch_loss)

        # VALIDATION
        sentinel2_model.eval()
        sentinel1_model.eval()
        downstream_model.eval()
        s1_classifier.eval()
        s2_classifier.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                images_s2, images_s1, labels = batch
                images_s2 = images_s2.to(device)
                images_s1 = images_s1.to(device)
                labels = labels.to(device)

                feature_s1 = sentinel1_model(images_s1)
                s1_classifier_preds = s1_classifier(feature_s1)
                s1_downstream_logits = downstream_model(feature_s1)
                s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
                feature_s2 = sentinel2_model(images_s2)
                s2_classifier_preds = s2_classifier(feature_s2)
                s2_downstream_logits = downstream_model(feature_s2)
                s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
                shared_network_loss = s1_downstream_loss + s2_downstream_loss

                s1 = torch.where(labels == 1, s1_classifier_preds, 1 - s1_classifier_preds)
                s2 = torch.where(labels == 1, s2_classifier_preds, 1 - s2_classifier_preds)
                cmfl = cmfl_loss(s1, s2) + cmfl_loss(s2, s1)
                loss = (shared_network_loss + cmfl)/2

                val_loss += loss.item()

        val_loss /= 2*len(val_loader.dataset)
        val_losses.append(val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    plt.figure(figsize=(8, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()
    plt.show()

def train_cmfl_model(sentinel2_model, sentinel1_model,
                s1_classifier, s2_classifier, downstream_model, optimizer,
                dataloader, device, num_epochs=35):
    bce_loss_fn = nn.BCEWithLogitsLoss()
    for epoch in range(num_epochs):
        sentinel2_model.train()
        sentinel1_model.train()
        downstream_model.train()
        s1_classifier.train()
        s2_classifier.train()
        epoch_loss = 0.0
        for batch in tqdm(dataloader):
            images_s2, images_s1, labels = batch
            images_s2 = images_s2.to(device)
            images_s1 = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            feature_s1 = sentinel1_model(images_s1)
            s1_classifier_preds = s1_classifier(feature_s1)
            s1_downstream_logits = downstream_model(feature_s1)
            s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
            feature_s2 = sentinel2_model(images_s2)
            s2_classifier_preds = s2_classifier(feature_s2)
            s2_downstream_logits = downstream_model(feature_s2)
            s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
            shared_network_loss = s1_downstream_loss + s2_downstream_loss

            s1 = torch.where(labels == 1, s1_classifier_preds, 1 - s1_classifier_preds)
            s2 = torch.where(labels == 1, s2_classifier_preds, 1 - s2_classifier_preds)
            cmfl = cmfl_loss(s1, s2) + cmfl_loss(s2, s1)
            loss = (shared_network_loss + cmfl)/2
            loss.backward()

            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= 2*len(dataloader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

def train_model_with_val(sentinel2_model, sentinel1_model, downstream_model, optimizer,
                         dataset, device, val_split=0.2, num_epochs=18, batch_size=32):
    bce_loss_fn = nn.BCEWithLogitsLoss()
    train_losses = []
    val_losses = []

    dataset_size = len(dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(num_epochs):
        # TRAINING
        sentinel2_model.train()
        sentinel1_model.train()
        downstream_model.train()
        epoch_loss = 0
        for batch in tqdm(train_loader):
            images_s2, images_s1, labels = batch
            images_s2 = images_s2.to(device)
            images_s1 = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            feature_s1 = sentinel1_model(images_s1)
            s1_downstream_logits = downstream_model(feature_s1)
            s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
            feature_s2 = sentinel2_model(images_s2)
            s2_downstream_logits = downstream_model(feature_s2)
            s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
            shared_network_loss = s1_downstream_loss + s2_downstream_loss
            shared_network_loss.backward()

            optimizer.step()
            epoch_loss += shared_network_loss.item()

        # VALIDATION
        sentinel2_model.eval()
        sentinel1_model.eval()
        downstream_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                images_s2, images_s1, labels = batch
                images_s2 = images_s2.to(device)
                images_s1 = images_s1.to(device)
                labels = labels.to(device)

                feature_s1 = sentinel1_model(images_s1)
                s1_downstream_logits = downstream_model(feature_s1)
                s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
                feature_s2 = sentinel2_model(images_s2)
                s2_downstream_logits = downstream_model(feature_s2)
                s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
                shared_network_loss = s1_downstream_loss + s2_downstream_loss

                val_loss += shared_network_loss.item()

        val_loss /= 2*len(val_loader.dataset)
        val_losses.append(val_loss)
        epoch_loss /= 2*len(train_loader.dataset)
        train_losses.append(epoch_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    plt.figure(figsize=(8, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()
    plt.show()


def train_model(sentinel2_model, sentinel1_model, downstream_model, optimizer,
                dataloader, device, num_epochs=12):

    bce_loss_fn = nn.BCEWithLogitsLoss()
    for epoch in range(num_epochs):
        sentinel2_model.train()
        sentinel1_model.train()
        downstream_model.train()
        epoch_loss = 0.0
        for batch in tqdm(dataloader):
            images_s2, images_s1, labels = batch
            images_s2 = images_s2.to(device)
            images_s1 = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            feature_s1 = sentinel1_model(images_s1)
            s1_downstream_logits = downstream_model(feature_s1)
            s1_downstream_loss = bce_loss_fn(s1_downstream_logits, labels)
            feature_s2 = sentinel2_model(images_s2)
            s2_downstream_logits = downstream_model(feature_s2)
            s2_downstream_loss = bce_loss_fn(s2_downstream_logits, labels)
            shared_network_loss = s2_downstream_loss + s1_downstream_loss
            shared_network_loss.backward()
            optimizer.step()
            epoch_loss += shared_network_loss.item()

        epoch_loss /= 2*len(dataloader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

def train_single_modality_with_val(sentinel_model, downstream_model, optimizer,
                         dataset, device, s2_flag, val_split=0.2, num_epochs=25, batch_size=32):
    bce_loss_fn = nn.BCEWithLogitsLoss()
    train_losses = []
    val_losses = []
    dataset_size = len(dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(num_epochs):
        # TRAINING
        sentinel_model.train()
        downstream_model.train()
        epoch_loss = 0.0
        for batch in tqdm(train_loader):
            images_s2, images_s1, labels = batch
            if s2_flag:
                images = images_s2.to(device)
            else:
                images = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            features = sentinel_model(images)
            downstream_logits = downstream_model(features)
            downstream_loss = bce_loss_fn(downstream_logits, labels)
            downstream_loss.backward()
            optimizer.step()
            epoch_loss += downstream_loss.item()

        epoch_loss /= 2*len(train_loader.dataset)
        train_losses.append(epoch_loss)

        # VALIDATION
        sentinel_model.eval()
        downstream_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                images_s2, images_s1, labels = batch
                if s2_flag:
                    images = images_s2.to(device)
                else:
                    images = images_s1.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                features = sentinel_model(images)
                downstream_logits = downstream_model(features)
                downstream_loss = bce_loss_fn(downstream_logits, labels)

                val_loss += downstream_loss.item()

        val_loss /= 2*len(val_loader.dataset)
        val_losses.append(val_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    plt.figure(figsize=(8, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()
    plt.show()

def train_model_single_modality(sentinel_model, downstream_model, optimizer,
                dataloader, device, s2_flag, num_epochs=12):
    bce_loss_fn = nn.BCEWithLogitsLoss()
    for epoch in range(num_epochs):
        sentinel_model.train()
        downstream_model.train()
        epoch_loss = 0.0
        for batch in tqdm(dataloader):
            images_s2, images_s1, labels = batch
            if s2_flag:
                images = images_s2.to(device)
            else:
                images = images_s1.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            features = sentinel_model(images)
            downstream_logits = downstream_model(features)
            downstream_loss = bce_loss_fn(downstream_logits, labels)
            downstream_loss.backward()
            optimizer.step()
            epoch_loss += downstream_loss.item()

        epoch_loss /= 2*len(dataloader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

def eval_model(extracter, classifier, data_loader, s2_flag):
    extracter.eval()
    classifier.eval()
    bce_loss_fn = nn.BCEWithLogitsLoss()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for batch in tqdm(data_loader):
            images_s2, images_s1, labels = batch
            labels = labels.to(device)
            if s2_flag:
                images_s2 = images_s2.to(device)
                feature = extracter(images_s2)
            else:
                images_s1 = images_s1.to(device)
                feature = extracter(images_s1)
            outputs = classifier(feature)
            probabilities = torch.sigmoid(outputs)
            predictions = (probabilities > 0.3).float()
            all_labels.append(labels.cpu().numpy())
            all_predictions.append(predictions.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_predictions = np.concatenate(all_predictions, axis=0)
    print("all predictions shape: ", all_predictions.shape)
    print("all labels shape: ", all_labels.shape)
    f1_macro = f1_score(all_labels, all_predictions, average='macro')
    f1_micro = f1_score(all_labels, all_predictions, average='micro')
    print(f'F1-Macro Score: {f1_macro:.3f}')
    print(f'F1-Micro Score: {f1_micro:.3f}')
    f1 = f1_score(all_labels, all_predictions, average=None)
    print("F1 Scores: " + ", ".join([f"{val:.3f}" for val in f1]))

## Loading and Testing

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)
sentinel2_model = s2_feature_extraction().to(device)
sentinel1_model = s1_feature_extraction().to(device)
downstream_model = downstream_classification().to(device)

In [ ]:
sentinel1_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/S1_S1_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/S1_downstream_model.pth'))
eval_model(sentinel1_model, downstream_model, Testloader, s2_flag=False)

In [ ]:
sentinel1_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/15_S1_S1_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/15_S1_downstream_model.pth'))
eval_model(sentinel1_model, downstream_model, Testloader, s2_flag=False)

In [ ]:
sentinel2_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/S2_S2_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/S2_downstream_model.pth'))
eval_model(sentinel2_model, downstream_model, Testloader, s2_flag=True)

In [ ]:
sentinel2_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/15_S2_S2_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/15_S2_downstream_model.pth'))
eval_model(sentinel2_model, downstream_model, Testloader, s2_flag=True)

In [ ]:
visualize_s2_sample_prediction(Testdataset, device, sentinel2_model, downstream_model, 12000)

In [ ]:
sentinel1_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/20_CM_S1_extractor.pth'))
sentinel2_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/20_CM_S2_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/CM/20_CM_downstream_model.pth'))
eval_model(sentinel1_model, downstream_model, Testloader, s2_flag=False)
print("\nNow S2 \n")
eval_model(sentinel2_model, downstream_model, Testloader, s2_flag=True)

In [ ]:
sentinel1_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/N_CMFL/S1_extractor.pth'))
sentinel2_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/N_CMFL/S2_extractor.pth'))
downstream_model.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/N_CMFL/downstream_model.pth'))
eval_model(sentinel1_model, downstream_model, Testloader, s2_flag=False)
print("\nNow S2 \n")
eval_model(sentinel2_model, downstream_model, Testloader, s2_flag=True)

In [ ]:
#testing a single training step
for i, batch in enumerate(Testloader):
      if i == 1:
        break
      else:
          images_s2, images_s1, labels = batch
          images_s2 = images_s2.to(device)
          images_s1 = images_s1.to(device)
          labels = labels.to(device)
          print("S1 shape: ", images_s1.shape)
          print("S2 shape: ", images_s2.shape)
          print("Labels shape: ", labels.shape)

          s1_sample = images_s1[30]
          s1_sample = s1_sample.squeeze().to('cpu')
          s1_rgb = np.dstack(np.array([s1_sample[0], s1_sample[1], s1_sample[0]- s1_sample[1]]))
          s2_sample = images_s2[30]
          s2_sample = s2_sample.squeeze().to('cpu')
          s2_rgb = np.dstack(np.array([s2_sample[3], s2_sample[2], s2_sample[1]]))

          fig, axes = plt.subplots(1, 2, figsize=(10, 5))

          axes[0].imshow(normalize_to_rgb(s1_rgb))
          axes[0].set_title('Sentinel-1')
          axes[0].axis('off')

          axes[1].imshow(normalize_to_rgb(s2_rgb))
          axes[1].set_title('Sentinel-2')
          axes[1].axis('off')

          plt.tight_layout()
          plt.show()



## Training and Evaluating Cross Modal



In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)
sentinel2_model = s2_feature_extraction().to(device)
sentinel1_model = s1_feature_extraction().to(device)
downstream_model = downstream_classification().to(device)
sentinel2_model.apply(init_weights)
sentinel1_model.apply(init_weights)
downstream_model.apply(init_weights)

optimizer = optim.Adam(list(sentinel2_model.parameters()) +
                       list(sentinel1_model.parameters()) +
                       list(downstream_model.parameters()), lr=0.001, weight_decay=0.00001)
s1_optimizer = optim.Adam(list(sentinel1_model.parameters()) +
                       list(downstream_model.parameters()), lr=0.001, weight_decay=0.00001)
s2_optimizer = optim.Adam(list(sentinel2_model.parameters()) +
                       list(downstream_model.parameters()), lr=0.001, weight_decay=0.00001)

In [ ]:
train_single_modality_with_val(sentinel1_model, downstream_model, s1_optimizer,
                            Traindataset, device, s2_flag=False, num_epochs = 30)

In [ ]:
train_model_single_modality(sentinel1_model, downstream_model, s1_optimizer,
                            Trainloader, device, s2_flag=False, num_epochs=20)

In [ ]:
train_single_modality_with_val(sentinel2_model, downstream_model, s2_optimizer,
                            Traindataset, device, s2_flag=True, num_epochs = 30)

In [ ]:
train_model_single_modality(sentinel2_model, downstream_model, s2_optimizer,
                            Trainloader, device, s2_flag=True, num_epochs=20)

In [ ]:
visualize_s2_sample_prediction(Testdataset, device, sentinel2_model, downstream_model, 500)

In [ ]:
train_model_with_val(sentinel2_model, sentinel1_model, downstream_model, optimizer,
                     Traindataset, device, val_split=0.2, num_epochs=40, batch_size=32)

In [ ]:
train_model(sentinel2_model, sentinel1_model, downstream_model, optimizer,
            Trainloader, device, num_epochs=30)

In [ ]:
visualize_s2_sample_prediction(Testdataset, device, sentinel2_model, downstream_model, 600)

In [ ]:
#torch.save(sentinel1_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S1_extractor.pth')
torch.save(sentinel2_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S2_extractor.pth')
#torch.save(downstream_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S1_downstream_model.pth')
torch.save(sentinel2_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S2_downstream_model.pth')

In [ ]:
torch.save(sentinel1_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S1_extractor.pth')
torch.save(sentinel2_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S2_extractor.pth')
torch.save(downstream_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_downstream_model.pth')

## Training and Evaluating Cross Modal with Focal Loss

In [ ]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)
sentinel2_model = s2_feature_extraction().to(device)
sentinel1_model = s1_feature_extraction().to(device)
s1_classifier = individual_classification().to(device)
s2_classifier = individual_classification().to(device)
downstream_model = downstream_classification().to(device)
sentinel2_model.apply(init_weights)
sentinel1_model.apply(init_weights)
downstream_model.apply(init_weights)

optimizer = optim.Adam(list(sentinel2_model.parameters()) +
                       list(sentinel1_model.parameters()) +
                       list(s1_classifier.parameters()) +
                       list(s2_classifier.parameters()) +
                       list(downstream_model.parameters()), lr=0.001, weight_decay=0.00001)

In [ ]:
train_cmfl_model_with_val(sentinel2_model, sentinel1_model, s1_classifier, s2_classifier, downstream_model, optimizer,
                     Traindataset, device, val_split=0.2, num_epochs=40, batch_size=32)

In [ ]:
train_cmfl_model(sentinel2_model, sentinel1_model, s1_classifier, s2_classifier, downstream_model, optimizer,
            Trainloader, device, num_epochs=40)

In [ ]:
visualize_s2_sample_prediction(Testdataset, device, sentinel2_model, downstream_model, 600)

In [ ]:
torch.save(sentinel1_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CMFL_S1_extractor.pth')
torch.save(sentinel2_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CMFL_S2_extractor.pth')
torch.save(downstream_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CMFL_downstream_model.pth')
torch.save(sentinel1_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_s1_classifier.pth')
torch.save(sentinel2_model.state_dict(), '/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_s2_classifier.pth')

#UMAP

In [ ]:
!pip install umap-learn

In [ ]:
import umap
import seaborn as sns

# Extract Features and Labels (currently for cross-modal feature extraction models)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
s2_feature_extractor = s2_feature_extraction().to(device)
s2_feature_extractor.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S2_extractor.pth'))
s1_feature_extractor = s1_feature_extraction().to(device)
s1_feature_extractor.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S1_extractor.pth'))

s1_features = []
s2_features = []
labels = []
with torch.no_grad():
    for batch in Trainloader:
        images_s2, images_s1, targets = batch
        images_s1 = images_s1.to(device)
        images_s2 = images_s2.to(device)
        features_s1 = s1_feature_extractor(images_s1)
        features_s2 = s2_feature_extractor(images_s2)
        s1_features.append(features_s1.cpu().numpy())
        s2_features.append(features_s2.cpu().numpy())
        labels.append(targets.cpu().numpy())

s1_features = np.concatenate(s1_features)  # Shape: (num_samples, 2048)
s2_features = np.concatenate(s2_features)
labels = np.concatenate(labels)  # Shape: (num_samples, 19)

In [ ]:
class_points_s1 = s1_features[labels[:, 18] == 1] # 18: Marine Waters
class_points_s2 = s2_features[labels[:, 18] == 1]
print(class_points_s1.shape)
print(class_points_s2.shape)
all_class_points = np.concatenate([class_points_s1, class_points_s2], axis = 0)
print(all_class_points.shape)

# Step 2: Perform UMAP on Entire Dataset
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean', random_state=42)
all_embeddings = umap_model.fit_transform(all_class_points)
s1_embeddings = all_embeddings[:class_points_s1.shape[0]]
s2_embeddings = all_embeddings[class_points_s1.shape[0]:]
print("2-d Marine Waters s1 feature vector count: ", s1_embeddings.shape)
print("2-d Marine Waters s2 feature vector count: ", s2_embeddings.shape)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(s1_embeddings[:, 0], s1_embeddings[:, 1], c = "blue", s=10, alpha=0.7, label = "Sentinel-1")
plt.scatter(s2_embeddings[:, 0], s2_embeddings[:, 1], c= "orange", s=10, alpha=0.7, label = "Sentinel-2")
plt.legend(markerscale = 3, loc = "lower right")
plt.title("Marine Waters Class in Cross-Modal Network")
plt.xlim(min(s1_embeddings[:, 0].min(), s2_embeddings[:, 0].min()) - 1, max(s1_embeddings[:, 0].max(), s2_embeddings[:, 0].max()) + 1)
plt.ylim(min(s1_embeddings[:, 1].min(), s2_embeddings[:, 1].min()) - 1, max(s1_embeddings[:, 1].max(), s2_embeddings[:, 1].max()) + 1)
plt.grid(True)
plt.show()

In [ ]:
import umap
import seaborn as sns

# Extract Features and Labels (change the path for other networks)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

baseline_feature_extractor = s2_feature_extraction().to(device)
baseline_feature_extractor.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S2_extractor.pth'))

baseline_features = []
labels = []
with torch.no_grad():
    for batch in Trainloader:
        images_s2, images_s1, targets = batch
        images_s2 = images_s2.to(device)
        features = baseline_feature_extractor(images_s2)
        baseline_features.append(features.cpu().numpy())
        labels.append(targets.cpu().numpy())

baseline_features = np.concatenate(baseline_features)
labels = np.concatenate(labels)

# Step 2: Perform UMAP on Entire Dataset
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean', random_state=42)
features_2d = umap_model.fit_transform(baseline_features)


In [ ]:
num_classes = labels.shape[1]
palette = sns.color_palette("hsv", num_classes)  # Generate colors for each class

In [ ]:
class_points_0 = features_2d[labels[:, 0] == 1]  # 2D embeddings for the selected class Urban Fabric
class_points_18 = features_2d[labels[:, 18] == 1]  # Marine Waters
class_points_8 = features_2d[labels[:, 8] == 1] # Broad Leaved Forest
plt.figure(figsize=(8, 6))
plt.scatter(class_points_8[:, 0], class_points_8[:, 1], c=[palette[8]], s=10, alpha=0.7, label = "Broad Leaved Forest")
plt.scatter(class_points_18[:, 0], class_points_18[:, 1], c=[palette[18]], s=10, alpha=0.7, label = "Marine Waters")
plt.scatter(class_points_0[:, 0], class_points_0[:, 1], c=[palette[0]], s=10, alpha=0.7, label = "Urban Fabric")
plt.legend(markerscale = 3)
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.xlim(features_2d[:, 0].min() - 1, features_2d[:, 0].max() + 1) ### ensure same coordinate for each class
plt.ylim(features_2d[:, 1].min() - 1, features_2d[:, 1].max() + 1)
plt.grid(True)
plt.show()

In [ ]:
class_points_16 = features_2d[labels[:, 16] == 1]  # Coastal Wetlands
class_points_10 = features_2d[labels[:, 10] == 1]  # Mixed Forest
class_points_2 = features_2d[labels[:, 2] == 1] # Arable Land
plt.figure(figsize=(8, 6))
plt.scatter(class_points_2[:, 0], class_points_2[:, 1], c=[palette[2]], s=10, alpha=0.7, label = "Arable Land")
plt.scatter(class_points_10[:, 0], class_points_10[:, 1], c=[palette[10]], s=10, alpha=0.7, label = "Mixed Forest")
plt.scatter(class_points_16[:, 0], class_points_16[:, 1], c=[palette[16]], s=10, alpha=0.7, label = "Coastal Wetlands")
plt.legend(markerscale = 3, loc = "lower right")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.xlim(features_2d[:, 0].min() - 1, features_2d[:, 0].max() + 1) ### ensure same coordinate for each class
plt.ylim(features_2d[:, 1].min() - 1, features_2d[:, 1].max() + 1)
plt.grid(True)
plt.show()

# Pearson Correlation

In [ ]:
# Function to compute Pearson correlation for two vectors (per-sample)
def pearson_correlation(x, y):
    mean_x = x.mean()
    mean_y = y.mean()
    numerator = torch.sum((x - mean_x) * (y - mean_y))
    denominator = torch.sqrt(torch.sum((x - mean_x) ** 2) * torch.sum((y - mean_y) ** 2))
    return numerator / denominator

# BASELINE
feature_extractor_s2 = s2_feature_extraction().to(device)
feature_extractor_s2.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S2_extractor.pth'))
feature_extractor_s1 = s1_feature_extraction().to(device)
feature_extractor_s1.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S1_extractor.pth'))
s2_classifier = downstream_classification().to(device)
s2_classifier.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S2_downstream_model.pth'))
s1_classifier = downstream_classification().to(device)
s1_classifier.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/S1_downstream_model.pth'))

# Initialize a list to store per-sample correlations
per_sample_correlations = []

# Loop through the test dataset
for batch in Testloader:
    with torch.no_grad():
        # Get images from the batch
        images_s2, images_s1, _ = batch
        images_s2 = images_s2.to(device)
        images_s1 = images_s1.to(device)

        features_s1 = feature_extractor_s1(images_s1)
        preds_s1 = s1_classifier(features_s1) # Predictions from Model 1 (for Sentinel-1)
        features_s2 = feature_extractor_s2(images_s2)
        preds_s2 = s2_classifier(features_s2)  # Predictions from Model 2 (for Sentinel-2)

        # Compute correlation for each sample in the batch
        for i in range(preds_s1.shape[0]):
            sample_preds_s1 = preds_s1[i]
            sample_preds_s2 = preds_s2[i]

            correlation = pearson_correlation(sample_preds_s1, sample_preds_s2)
            per_sample_correlations.append(correlation.item())  # Store the correlation for this sample

# Average the per-sample correlations
average_correlation = torch.mean(torch.tensor(per_sample_correlations))

print(f"Average Pearson Correlation between model predictions: {average_correlation.item()}")

In [ ]:
# Cross-Modal
feature_extractor_s2 = s2_feature_extraction().to(device)
feature_extractor_s2.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S2_extractor.pth'))
feature_extractor_s1 = s1_feature_extraction().to(device)
feature_extractor_s1.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_S1_extractor.pth'))
classifier = downstream_classification().to(device)
classifier.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights/CM_downstream_model.pth'))


# Initialize a list to store per-sample correlations
per_sample_correlations = []

# Loop through the test dataset
for batch in Testloader:
    with torch.no_grad():
        # Get images from the batch
        images_s2, images_s1, _ = batch
        images_s2 = images_s2.to(device)
        images_s1 = images_s1.to(device)

        features_s1 = feature_extractor_s1(images_s1)
        preds_s1 = classifier(features_s1) # Predictions from Model 1 (for Sentinel-1)
        features_s2 = feature_extractor_s2(images_s2)
        preds_s2 = classifier(features_s2)  # Predictions from Model 2 (for Sentinel-2)

        # Compute correlation for each sample in the batch
        for i in range(preds_s1.shape[0]):
            sample_preds_s1 = preds_s1[i]
            sample_preds_s2 = preds_s2[i]

            correlation = pearson_correlation(sample_preds_s1, sample_preds_s2)
            per_sample_correlations.append(correlation.item())  # Store the correlation for this sample

# Average the per-sample correlations
average_correlation = torch.mean(torch.tensor(per_sample_correlations))

print(f"Average Pearson Correlation between model predictions: {average_correlation.item()}")


In [ ]:
# Cross-Modal Focal Loss
feature_extractor_s2 = s2_feature_extraction().to(device)
feature_extractor_s2.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights2/CMFL_S2_extractor.pth'))
feature_extractor_s1 = s1_feature_extraction().to(device)
feature_extractor_s1.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights2/CMFL_S1_extractor.pth'))
classifier = downstream_classification().to(device)
classifier.load_state_dict(torch.load('/content/drive/My Drive/Bachelor_Thesis/Model/Model_Weights2/CMFL_downstream_model.pth'))

# Initialize a list to store per-sample correlations
per_sample_correlations = []

# Loop through the test dataset
for batch in Testloader:
    with torch.no_grad():
        # Get images from the batch
        images_s2, images_s1, label = batch
        images_s2 = images_s2.to(device)
        images_s1 = images_s1.to(device)

        features_s1 = feature_extractor_s1(images_s1)
        preds_s1 = classifier(features_s1) # Predictions from Model 1 (for Sentinel-1)
        features_s2 = feature_extractor_s2(images_s2)
        preds_s2 = classifier(features_s2)  # Predictions from Model 2 (for Sentinel-2)

        # Compute correlation for each sample in the batch
        for i in range(preds_s1.shape[0]):
            sample_preds_s1 = preds_s1[i]
            sample_preds_s2 = preds_s2[i]

            correlation = pearson_correlation(sample_preds_s1, sample_preds_s2)
            per_sample_correlations.append(correlation.item())  # Store the correlation for this sample

# Average the per-sample correlations
average_correlation = torch.mean(torch.tensor(per_sample_correlations))

print(f"Average Pearson Correlation between model predictions: {average_correlation.item()}")